### **SUMÁRIO**
>1. [01 - Pacotes Necessário](#pacotes)
>2. [02 - Carga de Dados](#carga-dados)
>3. [03 - Sobre o Modelo](#info)
>4. [04 - Conjunto de Dados de Treinamento e Teste](#datasets)
>5. [05 - Implamentação do Modelo](#impModelo)
    >>5.1 [05.1 - Função SIGMOID](#sigmoid)<br>
    >>5.2 [05.2 - Função para Inicializar os Parametros e Bias](#inicializar)<br>

***
<font size="6"><a id="pacotes">01 - Pacotes Necessários</a></font>
***

In [1]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
from sklearn.preprocessing import MinMaxScaler

In [2]:
cos_credentials={
  "apikey": "d9G8MTJTy4EdY0r5jVkCRDMSBFCKfmQaMRW8oOFI0eDD",
  "endpoints": "https://cos-service.bluemix.net/endpoints",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:cloud-object-storage:global:a/f819585dc4c43dfca0a953a8fc2635bb:a5bc02d4-816f-4d0b-8830-bd98421cb97f::",
  "iam_apikey_name": "auto-generated-apikey-b4c9dfbc-d2cd-429a-b3c5-4ad08f0735fa",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Manager",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/f819585dc4c43dfca0a953a8fc2635bb::serviceid:ServiceId-2e1053db-c54b-46d5-821c-85b692c0ac0e",
  "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/f819585dc4c43dfca0a953a8fc2635bb:a5bc02d4-816f-4d0b-8830-bd98421cb97f::"
}
auth_endpoint = 'https://iam.bluemix.net/oidc/token'
service_endpoint = 'https://s3.us-south.cloud-object-storage.appdomain.cloud'

In [3]:
cosFiles = ibm_boto3.client('s3',
                       ibm_api_key_id=cos_credentials['apikey'],
                       ibm_service_instance_id=cos_credentials['resource_instance_id'],
                       ibm_auth_endpoint=auth_endpoint,
                       config=Config(signature_version='oauth'),
                       endpoint_url=service_endpoint)

***
<font size="6"><a id="carga-dados">02 - Funções</a></font>
***

<div class="alert alert-block alert-success">
<b>Descrição: </b> Area destinada para realização do upload Cosfile "04work-crm" 
</div>

In [4]:
def SaveAndUploadFiles(fileName: str, dfToSave: pd.DataFrame):
    
    print('*'*50)
    print('*   Salvar Arquivo {} Localmente     *'.format(fileName))
    print('*'*50)
    dfToSave.to_csv(path_or_buf=fileName,index=False)
    
    print('*'*50)
    print('*      Executar Upload do Arquivo {} para o COS     *'.format(fileName))
    print('*'*50)
    
    cosFiles.upload_file(Filename=fileName,Bucket='04work-crm',Key=fileName)
    
    print('*'*50)
    print('*    Upload do Arquivo {} para o COS executado       *'.format(fileName))
    print('*'*50)

***
<font size="6"><a id="carga-dados">02 - Carga de Dados</a></font>
***

<div class="alert alert-block alert-success">
<b>Descrição: </b> O código a seguir acessa um arquivo no IBM Cloud Object Storage.<br> <b> <font color = "red"> Importante: </b> </font> Em caso de comparilhamento do notebook, remover as linhas com as marcações <b> "credenciais".</b>
</div>

In [5]:
#Carrega os dados de treinamento "base_refin_meio_a_meio_V2.csv"


def __iter__(self): return 0


client_a5bc02d4816f4d0b8830bd98421cb97f = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx', #credenciais removidas por motivo de segurança
    ibm_auth_endpoint="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx", #credenciais removidas por motivo de segurança
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_a5bc02d4816f4d0b8830bd98421cb97f.get_object(Bucket='crmmesadeperformanceii-donotdelete-pr-aa5r1ubmewwiyn',Key='base_refin_meio_a_meio_V2.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data = pd.read_csv(body)
df_data.head()


,NRO_CONTRATO,CNPJ_CPF,STATUS,TAXA,QUANT_PRESTACAO,VALOR_FINANCIADO,VALOR_PRESTACAO,DURACAO_CONTRATO,CANAL,GRPCONV,FLAG_REFIN,FLAG_SE_EH_REFIN
0,121419979,99999153100,E,1.392285,46,1960.87,58.90,88,CORRESPONDENTE,A-INSS,1,0
1,107819329,99995786672,E,2.377136,96,5035.75,138.25,544,CORRESPONDENTE,E-OUTROS,1,1
2,120615685,99994380834,E,2.369943,96,2664.11,72.16,498,CORRESPONDENTE,B-SIAPE,1,0
3,114315462,99993040100,E,2.099993,72,2940.29,81.37,771,CORRESPONDENTE,A-INSS,0,1
4,120089169,99990865868,E,1.957235,95,49245.60,1175.69,271,OUTROS,E-OUTROS,1,1


In [6]:
df_data.dtypes

NRO_CONTRATO          int64
CNPJ_CPF              int64
STATUS               object
TAXA                float64
QUANT_PRESTACAO       int64
VALOR_FINANCIADO    float64
VALOR_PRESTACAO     float64
DURACAO_CONTRATO      int64
CANAL                object
GRPCONV              object
FLAG_REFIN            int64
FLAG_SE_EH_REFIN      int64
dtype: object


***
<font size="6"><a id="info">03 - Tratamento dos Dados</a></font>
***

***
<font size="4"><a id="info">    03.1 - codificação one-hot</a></font>
***

<div class="alert alert-block alert-success">
<b>Descrição: </b>  A técnica "DUMMIE" consiste em converter valores categóricos em um vetor numérico unidimensional, ou seja, o vetor resultante terá apenas um elemento igual a 1 e a 0.  
</div>

In [7]:
arOneHotEncoding = ['STATUS','CANAL','GRPCONV']
dfArquivoTreinamento = pd.get_dummies(df_data,columns=arOneHotEncoding)
dfArquivoTreinamento.head()

,NRO_CONTRATO,CNPJ_CPF,TAXA,QUANT_PRESTACAO,VALOR_FINANCIADO,VALOR_PRESTACAO,DURACAO_CONTRATO,FLAG_REFIN,FLAG_SE_EH_REFIN,STATUS_A,...,STATUS_E,CANAL_BPV,CANAL_CORRESPONDENTE,CANAL_INTERNET,CANAL_OUTROS,GRPCONV_A-INSS,GRPCONV_B-SIAPE,GRPCONV_C-EST MT,GRPCONV_D-EST SC,GRPCONV_E-OUTROS
0,121419979,99999153100,1.392285,46,1960.87,58.90,88,1,0,0,...,1,0,1,0,0,1,0,0,0,0
1,107819329,99995786672,2.377136,96,5035.75,138.25,544,1,1,0,...,1,0,1,0,0,0,0,0,0,1
2,120615685,99994380834,2.369943,96,2664.11,72.16,498,1,0,0,...,1,0,1,0,0,0,1,0,0,0
3,114315462,99993040100,2.099993,72,2940.29,81.37,771,0,1,0,...,1,0,1,0,0,1,0,0,0,0
4,120089169,99990865868,1.957235,95,49245.60,1175.69,271,1,1,0,...,1,0,0,0,1,0,0,0,0,1


***
<font size="6"><a id="info">03.2 - Normalização</a></font>
***

<div class="alert alert-block alert-success">
<b>Descrição:</b> São gerados valores entre 0 e 1 para as variaveis.
</div>

In [8]:
arNormalize = ['TAXA','QUANT_PRESTACAO','VALOR_FINANCIADO','VALOR_PRESTACAO','DURACAO_CONTRATO']
scaler = MinMaxScaler(feature_range=(0,1))

for eachCategory in arNormalize:
    dfArquivoTreinamento[eachCategory] = scaler.fit_transform(dfArquivoTreinamento[eachCategory].values.reshape(-1, 1))

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [9]:
dfArquivoTreinamento.head()

,NRO_CONTRATO,CNPJ_CPF,TAXA,QUANT_PRESTACAO,VALOR_FINANCIADO,VALOR_PRESTACAO,DURACAO_CONTRATO,FLAG_REFIN,FLAG_SE_EH_REFIN,STATUS_A,...,STATUS_E,CANAL_BPV,CANAL_CORRESPONDENTE,CANAL_INTERNET,CANAL_OUTROS,GRPCONV_A-INSS,GRPCONV_B-SIAPE,GRPCONV_C-EST MT,GRPCONV_D-EST SC,GRPCONV_E-OUTROS
0,121419979,99999153100,0.315140,0.468085,0.009895,0.013569,0.058945,1,0,0,...,1,0,1,0,0,1,0,0,0,0
1,107819329,99995786672,0.538059,1.000000,0.025711,0.031853,0.530507,1,1,0,...,1,0,1,0,0,0,0,0,0,1
2,120615685,99994380834,0.536430,1.000000,0.013512,0.016625,0.482937,1,0,0,...,1,0,1,0,0,0,1,0,0,0
3,114315462,99993040100,0.475328,0.744681,0.014933,0.018747,0.765253,0,1,0,...,1,0,1,0,0,1,0,0,0,0
4,120089169,99990865868,0.443015,0.989362,0.253108,0.270901,0.248190,1,1,0,...,1,0,0,0,1,0,0,0,0,1


***
<font size="6"><a id="info">03.3 - Remover Colunas não necessárias</a></font>
***

<div class="alert alert-block alert-success">
<b>Descrição: </b> São excluidas as colunas do tipo "numero de contrato" e "cpf" pois podem enviesar o modelo.
</div>

In [10]:
arColunsToRemove = ['NRO_CONTRATO','CNPJ_CPF']
dfArquivoTreinamento.drop(columns=arColunsToRemove, inplace=True,axis=1)

In [11]:
dfArquivoTreinamento.head()

,TAXA,QUANT_PRESTACAO,VALOR_FINANCIADO,VALOR_PRESTACAO,DURACAO_CONTRATO,FLAG_REFIN,FLAG_SE_EH_REFIN,STATUS_A,STATUS_C,STATUS_E,CANAL_BPV,CANAL_CORRESPONDENTE,CANAL_INTERNET,CANAL_OUTROS,GRPCONV_A-INSS,GRPCONV_B-SIAPE,GRPCONV_C-EST MT,GRPCONV_D-EST SC,GRPCONV_E-OUTROS
0,0.315140,0.468085,0.009895,0.013569,0.058945,1,0,0,0,1,0,1,0,0,1,0,0,0,0
1,0.538059,1.000000,0.025711,0.031853,0.530507,1,1,0,0,1,0,1,0,0,0,0,0,0,1
2,0.536430,1.000000,0.013512,0.016625,0.482937,1,0,0,0,1,0,1,0,0,0,1,0,0,0
3,0.475328,0.744681,0.014933,0.018747,0.765253,0,1,0,0,1,0,1,0,0,1,0,0,0,0
4,0.443015,0.989362,0.253108,0.270901,0.248190,1,1,0,0,1,0,0,0,1,0,0,0,0,1


***
<font size="6"><a id="info">05 - Salvar Arquivo de Treinamento</a></font>
***

<div class="alert alert-block alert-success">
<b>Descrição: </b>
    Area destina em realizar a alteração do nome da coluna target e a gravação do arquivo .csv
</div>

In [12]:
dfArquivoTreinamento.rename(columns={'FLAG_REFIN':'y'},inplace=True)
SaveAndUploadFiles('ArquivoTreinamento.csv',dfArquivoTreinamento)

**************************************************
*   Salvar Arquivo ArquivoTreinamento.csv Localmente     *
**************************************************
**************************************************
*      Executar Upload do Arquivo ArquivoTreinamento.csv para o COS     *
**************************************************
**************************************************
*    Upload do Arquivo ArquivoTreinamento.csv para o COS executado       *
**************************************************


In [13]:
dfArquivoTreinamento.drop(columns='y', inplace=True) 

In [14]:
dfcolunas=pd.DataFrame(data=dfArquivoTreinamento.columns,columns=['colunas'])

In [15]:
SaveAndUploadFiles('colunas.csv',dfcolunas)

**************************************************
*   Salvar Arquivo colunas.csv Localmente     *
**************************************************
**************************************************
*      Executar Upload do Arquivo colunas.csv para o COS     *
**************************************************
**************************************************
*    Upload do Arquivo colunas.csv para o COS executado       *
**************************************************
